In [3]:
import psycopg2
import pyodbc
import sqlite3
import pygrametl
from pygrametl.datasources import SQLSource, CSVSource
from pygrametl.tables import CachedDimension, FactTable

Opening of connections and creation of a ConnectionWrapper.
Creates a PEP 249 connection to the sales database. PARSE_DECLTYPES makes
sqlite3 return values with the types specified in the database's schema.

While SQLite is used in this guide, any RDBMS that provides a PEP 249
driver can be used with pygrametl. For example, SQLite can be replaced
with PostgreSQL by simply replacing sale_conn with following two lines.
sale_string = "host='localhost' dbname='sale' user='user' password='pass'"
sale_conn = psycopg2.connect(sale_string)

A connection is also created to the data warehouse. The connection is
then given to a ConnectionWrapper so it becomes implicitly shared between
all the pygrametl abstractions that needs it without being passed around
dw_string = "host='localhost' dbname='dw' user='dwuser' password='dwpass'"
dw_conn = psycopg2.connect(dw_string)

Although the ConnectionWrapper is shared automatically between pygrametl
abstractions, it is saved in a variable so the connection can be closed

In [4]:
# dw_conn_wrapper = pygrametl.ConnectionWrapper(connection=dw_conn)

sale_conn = sqlite3.connect("sale.sqlite",
        detect_types=sqlite3.PARSE_DECLTYPES)

dw_conn =  pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=PROD-LPT-69\SQL;"
                      "Database=dw;"
                      "Trusted_Connection=yes;")
dw_conn_wrapper = pygrametl.ConnectionWrapper(connection=dw_conn)

In [5]:
# The location dimension stores the name of a location in the column city
# instead of in the column store as done in the input data from the sales
# database. By passing SQLSource a sequence of names matching the number of
# columns in the table it can automatically rename the columns
name_mapping = 'book', 'genre', 'city', 'date', 'sale'

# Extraction of rows from a database using a PEP 249 connection and SQL
query = "SELECT book, genre, store, date, sale FROM sale"
sale_source = SQLSource(connection=sale_conn, query=query,
        names=name_mapping)

# Extraction of rows from a CSV file does not require a PEP 249 connection,
# just an open file handler. pygrametl uses Python's DictReader for CSV
# files and assumes the header of the CSV file contains the name of each
# column. When using CSVSource it is very important to convert the values
# to the correct type before inserting them into a table through pygrametl
region_file_handle = open('region.csv', 'r', 16384)
region_source = CSVSource(f=region_file_handle, delimiter=',')

An instance of CachedDimension is created for each dimension in the data
warehouse. CachedDimension uses a local cache to significantly reduce the
number of requests issued to the RDBMS. CachedDimension should generally
be used instead of Dimension unless the higher memory consumption causes
problems. For each dimension, the name of the database table, the table's
primary key, and the table's non-key columns (attributes) are given. In
addition, for the location dimension the subset of the attributes that
should be used to lookup the primary key are given. As mentioned in the
beginning of this guide, using named parameters is strongly encouraged

In [6]:
book_dimension = CachedDimension(
        name='book',
        key='bookid',
        attributes=['book', 'genre'])

time_dimension = CachedDimension(
        name='time',
        key='timeid',
        attributes=['day', 'month', 'year'])

location_dimension = CachedDimension(
        name='location',
        key='locationid',
        attributes=['city', 'region'],
        lookupatts=['city'])

# A single instance of FactTable is created for the data warehouse's single
# fact table. It is created with the name of the table, a list of columns
# constituting the primary key of the fact table, and a list of measures
fact_table = FactTable(
        name='facttable',
        keyrefs=['bookid', 'locationid', 'timeid'],
        measures=['sale'])

In [7]:
# A normal Python function is used to split the date into its parts
def split_date(row):
    """Splits a date represented by a datetime into its three parts"""

    # First the datetime object is extracted from the row dictionary
    date = row['date']

    # Then each part is reassigned to the row dictionary. It can then be
    # accessed by the caller as the row is a reference to the dict object
    row['year'] = date.year
    row['month'] = date.month
    row['day'] = date.day

In [8]:
# The location dimension is loaded from the CSV file
[location_dimension.insert(row) for row in region_source]

# The file handle for the CSV file can then be closed
region_file_handle.close()

# Each row in the sales database is iterated through and inserted
for row in sale_source:

    # Each row is passed to the date split function for splitting
    split_date(row)

    # Lookups are performed to find the key in each dimension for the fact
    # and if the data is not there, it is inserted from the sales row
    row['bookid'] = book_dimension.ensure(row)
    row['timeid'] = time_dimension.ensure(row)
    
    # The location dimension is pre-filled, so a missing row is an error
    row['locationid'] = location_dimension.lookup(row)
    if not row['locationid']:
        row['locationid'] = 1
#         raise ValueError("city was not present in the location dimension")

    print(row)
    # The row can then be inserted into the fact table
    fact_table.insert(row)

{'book': 'Nineteen Eighty-Four', 'genre': 'Novel', 'city': 'Aalborg', 'date': datetime.date(2005, 8, 5), 'sale': 50, 'year': 2005, 'month': 8, 'day': 5, 'bookid': 1, 'timeid': 1, 'locationid': 1}
{'book': 'Calvin and Hobbes One', 'genre': 'Comic', 'city': 'Aalborg', 'date': datetime.date(2005, 8, 5), 'sale': 25, 'year': 2005, 'month': 8, 'day': 5, 'bookid': 2, 'timeid': 1, 'locationid': 1}
{'book': 'The Silver Spoon', 'genre': 'Cookbook', 'city': 'Aalborg', 'date': datetime.date(2005, 8, 14), 'sale': 5, 'year': 2005, 'month': 8, 'day': 14, 'bookid': 3, 'timeid': 2, 'locationid': 1}
{'book': 'The Silver Spoon', 'genre': 'Cookbook', 'city': 'Odense', 'date': datetime.date(2005, 9, 1), 'sale': 7, 'year': 2005, 'month': 9, 'day': 1, 'bookid': 3, 'timeid': 3, 'locationid': 26}
{'book': 'Mr. Frank, the Underground Mail-Agent', 'genre': 'Parody', 'city': 'Slagelse', 'date': datetime.date(2005, 2, 16), 'sale': 84, 'year': 2005, 'month': 2, 'day': 16, 'bookid': 4, 'timeid': 4, 'locationid': 39}

{'book': 'The Anatomy Lesson', 'genre': 'Fiction', 'city': 'Nakskov', 'date': datetime.date(2005, 3, 1), 'sale': 72, 'year': 2005, 'month': 3, 'day': 1, 'bookid': 81, 'timeid': 70, 'locationid': 36}
{'book': 'City of the Chasch', 'genre': 'Science Fiction', 'city': 'Skagen', 'date': datetime.date(2005, 3, 28), 'sale': 68, 'year': 2005, 'month': 3, 'day': 28, 'bookid': 82, 'timeid': 71, 'locationid': 5}
{'book': "Une Page d'amour", 'genre': 'Novel', 'city': 'Odense', 'date': datetime.date(2005, 7, 5), 'sale': 42, 'year': 2005, 'month': 7, 'day': 5, 'bookid': 83, 'timeid': 72, 'locationid': 26}
{'book': 'The Mad Man', 'genre': 'Fiction', 'city': 'Tønder', 'date': datetime.date(2005, 5, 8), 'sale': 93, 'year': 2005, 'month': 5, 'day': 8, 'bookid': 84, 'timeid': 73, 'locationid': 1}
{'book': 'Cryptid Hunters', 'genre': 'Fantasy', 'city': 'Sønderborg', 'date': datetime.date(2005, 12, 2), 'sale': 61, 'year': 2005, 'month': 12, 'day': 2, 'bookid': 85, 'timeid': 74, 'locationid': 1}
{'book': '

{'book': 'Chorus Skating', 'genre': 'Speculative fiction', 'city': 'Tønder', 'date': datetime.date(2005, 6, 1), 'sale': 28, 'year': 2005, 'month': 6, 'day': 1, 'bookid': 128, 'timeid': 54, 'locationid': 1}
{'book': 'The Heart of the Warrior', 'genre': 'Science Fiction', 'city': 'Frederiksberg', 'date': datetime.date(2005, 1, 7), 'sale': 80, 'year': 2005, 'month': 1, 'day': 7, 'bookid': 129, 'timeid': 109, 'locationid': 8}
{'book': 'Kindred Spirits', 'genre': 'Fantasy', 'city': 'Aabenraa', 'date': datetime.date(2005, 12, 11), 'sale': 8, 'year': 2005, 'month': 12, 'day': 11, 'bookid': 130, 'timeid': 110, 'locationid': 20}
{'book': 'April Lady', 'genre': 'Mystery', 'city': 'Esbjerg', 'date': datetime.date(2005, 7, 16), 'sale': 97, 'year': 2005, 'month': 7, 'day': 16, 'bookid': 131, 'timeid': 111, 'locationid': 21}
{'book': 'The Bladerunner', 'genre': 'Science Fiction', 'city': 'Sønderborg', 'date': datetime.date(2005, 11, 14), 'sale': 6, 'year': 2005, 'month': 11, 'day': 14, 'bookid': 132

{'book': 'My Education: A Book of Dreams', 'genre': 'Novel', 'city': 'Sønderborg', 'date': datetime.date(2005, 8, 8), 'sale': 65, 'year': 2005, 'month': 8, 'day': 8, 'bookid': 181, 'timeid': 30, 'locationid': 1}
{'book': 'Incest: From a Journal of Love', 'genre': 'Memoir', 'city': 'Haderslev', 'date': datetime.date(2005, 5, 23), 'sale': 87, 'year': 2005, 'month': 5, 'day': 23, 'bookid': 182, 'timeid': 131, 'locationid': 23}
{'book': 'London Blues', 'genre': 'Crime Fiction', 'city': 'Ribe', 'date': datetime.date(2005, 9, 18), 'sale': 17, 'year': 2005, 'month': 9, 'day': 18, 'bookid': 183, 'timeid': 141, 'locationid': 27}
{'book': 'The Chinese Bell Murders', 'genre': 'Mystery', 'city': 'Hillerød', 'date': datetime.date(2005, 5, 22), 'sale': 20, 'year': 2005, 'month': 5, 'day': 22, 'bookid': 184, 'timeid': 142, 'locationid': 1}
{'book': 'Counter-Clock World', 'genre': 'Science Fiction', 'city': 'Horsens', 'date': datetime.date(2005, 10, 6), 'sale': 62, 'year': 2005, 'month': 10, 'day': 6,

{'book': 'Explorers on the Moon', 'genre': 'Comic book', 'city': 'Aabenraa', 'date': datetime.date(2005, 6, 26), 'sale': 86, 'year': 2005, 'month': 6, 'day': 26, 'bookid': 230, 'timeid': 170, 'locationid': 20}
{'book': 'Thinks ...', 'genre': 'Fiction', 'city': 'Aabenraa', 'date': datetime.date(2005, 1, 21), 'sale': 30, 'year': 2005, 'month': 1, 'day': 21, 'bookid': 231, 'timeid': 134, 'locationid': 20}
{'book': 'Gather Together in My Name', 'genre': 'Autobiography', 'city': 'Sønderborg', 'date': datetime.date(2005, 9, 7), 'sale': 70, 'year': 2005, 'month': 9, 'day': 7, 'bookid': 232, 'timeid': 171, 'locationid': 1}
{'book': 'Little Altars Everywhere', 'genre': 'Anthology', 'city': 'Køge', 'date': datetime.date(2005, 8, 24), 'sale': 26, 'year': 2005, 'month': 8, 'day': 24, 'bookid': 233, 'timeid': 172, 'locationid': 1}
{'book': 'The Slave-girl from Jerusalem', 'genre': 'Historical fiction', 'city': 'Sønderborg', 'date': datetime.date(2005, 1, 31), 'sale': 100, 'year': 2005, 'month': 1, 

...

{'book': "Henderson's Boys: The Escape", 'genre': 'Thriller', 'city': 'Køge', 'date': datetime.date(2005, 2, 10), 'sale': 60, 'year': 2005, 'month': 2, 'day': 10, 'bookid': 4142, 'timeid': 64, 'locationid': 1}
{'book': "The King's Fifth", 'genre': "Children's literature", 'city': 'Svendborg', 'date': datetime.date(2005, 4, 3), 'sale': 75, 'year': 2005, 'month': 4, 'day': 3, 'bookid': 4143, 'timeid': 193, 'locationid': 29}
{'book': 'Blueberries for Sal', 'genre': 'Picture book', 'city': 'Roskilde', 'date': datetime.date(2005, 12, 4), 'sale': 46, 'year': 2005, 'month': 12, 'day': 4, 'bookid': 4144, 'timeid': 162, 'locationid': 38}
{'book': 'The Billion Dollar Ransom', 'genre': 'Mystery', 'city': 'Viborg', 'date': datetime.date(2005, 4, 22), 'sale': 15, 'year': 2005, 'month': 4, 'day': 22, 'bookid': 864, 'timeid': 17, 'locationid': 19}
{'book': 'Lord Jim', 'genre': 'Science Fiction', 'city': 'Frederikshavn', 'date': datetime.date(2005, 6, 17), 'sale': 37, 'year': 2005, 'month': 6, 'day': 

{'book': 'The Princes of the Golden Cage', 'genre': 'Speculative fiction', 'city': 'Silkeborg', 'date': datetime.date(2005, 11, 1), 'sale': 49, 'year': 2005, 'month': 11, 'day': 1, 'bookid': 4175, 'timeid': 285, 'locationid': 17}
{'book': 'A Star Called Henry', 'genre': 'Fiction', 'city': 'Svendborg', 'date': datetime.date(2005, 6, 20), 'sale': 30, 'year': 2005, 'month': 6, 'day': 20, 'bookid': 4176, 'timeid': 362, 'locationid': 29}
{'book': 'Mort', 'genre': 'Science Fiction', 'city': 'Maribo', 'date': datetime.date(2005, 3, 15), 'sale': 80, 'year': 2005, 'month': 3, 'day': 15, 'bookid': 4177, 'timeid': 177, 'locationid': 35}
{'book': 'Peace Breaks Out', 'genre': "Children's literature", 'city': 'Frederiksberg', 'date': datetime.date(2005, 2, 7), 'sale': 71, 'year': 2005, 'month': 2, 'day': 7, 'bookid': 4040, 'timeid': 271, 'locationid': 8}
{'book': 'Harnessing Peacocks', 'genre': 'Novel', 'city': 'Nykøbing Falster', 'date': datetime.date(2005, 2, 7), 'sale': 79, 'year': 2005, 'month':

{'book': 'No Logo', 'genre': 'Business', 'city': 'Tønder', 'date': datetime.date(2005, 9, 30), 'sale': 83, 'year': 2005, 'month': 9, 'day': 30, 'bookid': 4214, 'timeid': 331, 'locationid': 1}
{'book': 'Conflict of Interest', 'genre': 'Novel', 'city': 'Silkeborg', 'date': datetime.date(2005, 3, 29), 'sale': 52, 'year': 2005, 'month': 3, 'day': 29, 'bookid': 4215, 'timeid': 33, 'locationid': 17}
{'book': 'Birth of an Age', 'genre': 'Science Fiction', 'city': 'Sorø', 'date': datetime.date(2005, 1, 11), 'sale': 66, 'year': 2005, 'month': 1, 'day': 11, 'bookid': 4216, 'timeid': 273, 'locationid': 1}
{'book': 'Dangling Man', 'genre': 'Fiction', 'city': 'Vordingborg', 'date': datetime.date(2005, 7, 12), 'sale': 47, 'year': 2005, 'month': 7, 'day': 12, 'bookid': 3689, 'timeid': 206, 'locationid': 41}
{'book': 'The Stars Shine Down', 'genre': 'Thriller', 'city': 'Nyborg', 'date': datetime.date(2005, 3, 3), 'sale': 82, 'year': 2005, 'month': 3, 'day': 3, 'bookid': 4217, 'timeid': 98, 'locationid

{'book': '...And Call Me Conrad', 'genre': 'Science Fiction', 'city': 'Sorø', 'date': datetime.date(2005, 10, 13), 'sale': 52, 'year': 2005, 'month': 10, 'day': 13, 'bookid': 4251, 'timeid': 286, 'locationid': 1}
{'book': 'Atom Bomb Blues', 'genre': 'Science Fiction', 'city': 'Odense', 'date': datetime.date(2005, 12, 27), 'sale': 19, 'year': 2005, 'month': 12, 'day': 27, 'bookid': 4252, 'timeid': 304, 'locationid': 26}
{'book': 'Ilsa', 'genre': 'Novel', 'city': 'Vejle', 'date': datetime.date(2005, 3, 19), 'sale': 14, 'year': 2005, 'month': 3, 'day': 19, 'bookid': 4253, 'timeid': 148, 'locationid': 31}
{'book': 'The Anybodies', 'genre': "Children's literature", 'city': 'Fredericia', 'date': datetime.date(2005, 7, 19), 'sale': 33, 'year': 2005, 'month': 7, 'day': 19, 'bookid': 4254, 'timeid': 238, 'locationid': 22}
{'book': 'Escape Velocity', 'genre': 'Speculative fiction', 'city': 'Haderslev', 'date': datetime.date(2005, 5, 5), 'sale': 55, 'year': 2005, 'month': 5, 'day': 5, 'bookid': 4

{'book': 'To Serve Them All My Days', 'genre': 'Historical novel', 'city': 'Nyborg', 'date': datetime.date(2005, 12, 31), 'sale': 33, 'year': 2005, 'month': 12, 'day': 31, 'bookid': 4287, 'timeid': 318, 'locationid': 25}
{'book': '1635: The Eastern Front', 'genre': 'Alternate history', 'city': 'Ribe', 'date': datetime.date(2005, 9, 7), 'sale': 39, 'year': 2005, 'month': 9, 'day': 7, 'bookid': 4288, 'timeid': 171, 'locationid': 27}
{'book': 'Empire of the Sun', 'genre': 'Autobiographical novel', 'city': 'Maribo', 'date': datetime.date(2005, 5, 12), 'sale': 100, 'year': 2005, 'month': 5, 'day': 12, 'bookid': 4289, 'timeid': 62, 'locationid': 35}
{'book': 'The Boy Who Dared', 'genre': 'Historical fiction', 'city': 'Viborg', 'date': datetime.date(2005, 9, 7), 'sale': 19, 'year': 2005, 'month': 9, 'day': 7, 'bookid': 4290, 'timeid': 171, 'locationid': 19}
{'book': 'The Santaroga Barrier', 'genre': 'Science Fiction', 'city': 'Frederiksberg', 'date': datetime.date(2005, 1, 13), 'sale': 76, 'y

In [9]:
# The data warehouse connection is then ordered to commit and close
dw_conn_wrapper.commit()
dw_conn_wrapper.close()

# Finally, the connection to the sales database is closed
sale_conn.close()